# Welcome To The Galapagos Setup

The Galapagos Middleware presents an orchestration framework to create multi-FPGA and CPU networks and can be built on top of any device that has a Galapagos Hypervisor.

<a id='own_example'></a>

# Setup

## Setup Environment Variables

The following cells follow the initial setup. The first sets up the environment variables needed by the scripts below.

In [5]:
galapagos_path=/home/user/galapagos
cd $galapagos_path

vivado_version=2018.1
hls_version=2018.1
galapagos_board=adm-8k5-debug
vivado_path=/opt/Xilinx/Vivado
hls_path=/opt/Xilinx/Vivado_HLS/$hls_version

source init.sh $galapagos_path $vivado_path $hls_path $vivado_version $hls_version
galapagos-update-board $galapagos_board
export XILINX_VIVADO=$GALAPAGOS_VIVADO_PATH

SyntaxError: invalid syntax (<ipython-input-5-8fe8887190f0>, line 1)

## Make Middleware IP


Next we need to build the IP cores used by the middleware layer to connect different Galapagos objects

The middleware IP cores are shown in the following figure:

<img style="float: left;" src="fig/middleware_ip.png"/>  

In [2]:
cd $GALAPAGOS_PATH
make hlsmiddleware  

mkdir -p /home/user/galapagos/hlsBuild
make -C /home/user/galapagos/middleware hlsmiddleware
make[1]: Entering directory '/home/user/galapagos/middleware'
make -C /home/user/galapagos/middleware/hls
make[2]: Entering directory '/home/user/galapagos/middleware/hls'
for dir in galapagos_bridge/. router/. network_bridge_tcp/. network_bridge_raw/. network_bridge_ethernet/.; do \
	make -C $dir -f Makefile all; \
done
make[3]: Entering directory '/home/user/galapagos/middleware/hls/galapagos_bridge'
mkdir -p /home/user/galapagos/hlsBuild/adm-8k5-debug/ip/galapagos_bridge
vivado_hls /home/user/galapagos/middleware/hls/galapagos_bridge/generate_hls.tcl

****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2018.1 (64-bit)
  **** SW Build 2188600 on Wed Apr  4 18:39:19 MDT 2018
  **** IP Build 2185939 on Wed Apr  4 20:55:05 MDT 2018
    ** Copyright 1986-2018 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2018.1/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10

INFO: [BIND 205-100] Finished micro-architecture generation.
INFO: [HLS 200-111]  Elapsed time: 0.04 seconds; current allocated memory: 118.311 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-10] -- Generating RTL for module 'g2N_ingress'
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [RTGEN 206-100] Finished creating RTL model for 'g2N_ingress'.
INFO: [HLS 200-111]  Elapsed time: 0.08 seconds; current allocated memory: 118.552 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-10] -- Generating RTL for module 'g2N_egress'
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [RTGEN 206-100] Finished creating RTL model for 'g2N_egress'.
INFO: [HLS 200-111]  Elapsed time: 0.1 seconds; current allocated memory: 119.399 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------

 #define SSDM_OP_ATTR __attribute__ ((nothrow)) __attribute__((overloadable))
                                                                            ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:166:36: note: in expansion of macro 'SSDM_OP_ATTR'
     _uint1_ _ssdm_op_IfNbRead(...) SSDM_OP_ATTR;
                                    ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:153:76: warning: 'bitwidth' attribute directive ignored [-Wattributes]
 #define SSDM_OP_ATTR __attribute__ ((nothrow)) __attribute__((overloadable))
                                                                            ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:166:36: note: in expansion of macro 'SSDM_OP_ATTR'
     _uint1_ _ssdm_op_IfNbRead(...) SSDM_OP_ATTR;
                                    ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:153:76: warning: 'overloadable' attribute directive ignored [-Wattributes]
 #define SSDM_OP_ATTR __attribut

     _uint1_  _ssdm_StreamNbWrite(...) SSDM_OP_ATTR;
                                       ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:153:76: warning: 'overloadable' attribute directive ignored [-Wattributes]
 #define SSDM_OP_ATTR __attribute__ ((nothrow)) __attribute__((overloadable))
                                                                            ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:176:39: note: in expansion of macro 'SSDM_OP_ATTR'
     _uint1_  _ssdm_StreamCanRead(...) SSDM_OP_ATTR;
                                       ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:153:76: warning: 'bitwidth' attribute directive ignored [-Wattributes]
 #define SSDM_OP_ATTR __attribute__ ((nothrow)) __attribute__((overloadable))
                                                                            ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:176:39: note: in expansion of macro 'SSDM_OP_ATTR'
     _uint1_  _ssdm_

 #define SSDM_SPEC_ATTR __attribute__ ((nothrow)) __attribute__((overloadable))
                                                                              ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:195:33: note: in expansion of macro 'SSDM_SPEC_ATTR'
     void _ssdm_op_SpecPort(...) SSDM_SPEC_ATTR;
                                 ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:152:78: warning: 'overloadable' attribute directive ignored [-Wattributes]
 #define SSDM_SPEC_ATTR __attribute__ ((nothrow)) __attribute__((overloadable))
                                                                              ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:196:39: note: in expansion of macro 'SSDM_SPEC_ATTR'
     void _ssdm_op_SpecConnection(...) SSDM_SPEC_ATTR;
                                       ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:152:78: warning: 'overloadable' attribute directive ignored [-Wattributes]
 #define SSD

...

INFO: [IMPL 213-200] Port 'rxGalapagosBridge_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'rxGalapagosBridge_V.keep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-200] Port 'rxGalapagosBridge_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'rxData_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'rxData_V.keep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-200] Port 'rxData_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'rxMetaData_V_V' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'txStatus_V_V' is mapped to 'TDATA' by default.
INFO: [HLS 200-111] Finished generating all RTL models Time (s): cpu = 00:01:39 ; elapsed = 00:00:58 . Memory (MB): peak = 749.836 ; gain = 401.270 ; free physical = 40729 ; free virtual = 120043
INFO: [SYSC 207-301] Generating SystemC RTL for network_bridge_tcp.
INFO: [VHDL 208-304] Generating VHDL RTL for network_bridge_tcp.
INFO: [VLOG 209-307] Generating Verilo

INFO: [HLS 200-10] Checking synthesizability ...
INFO: [XFORM 203-602] Inlining function 'reverseEndian64_data' into 'raw_to_app' (/home/user/galapagos/middleware/hls/network_bridge_raw/src/raw_bridge.cpp:61) automatically.
INFO: [XFORM 203-602] Inlining function 'reverseEndian64_data' into 'app_to_raw' (/home/user/galapagos/middleware/hls/network_bridge_raw/src/raw_bridge.cpp:112) automatically.
INFO: [HLS 200-111] Finished Checking Synthesizability Time (s): cpu = 00:01:00 ; elapsed = 00:00:24 . Memory (MB): peak = 476.961 ; gain = 128.395 ; free physical = 40979 ; free virtual = 120315
INFO: [XFORM 203-1101] Packing variable 'to_app.V' (/home/user/galapagos/middleware/hls/network_bridge_raw/src/raw_bridge.cpp:154) into a 89-bit variable.
INFO: [XFORM 203-1101] Packing variable 'from_raw.V' (/home/user/galapagos/middleware/hls/network_bridge_raw/src/raw_bridge.cpp:155) into a 73-bit variable.
INFO: [XFORM 203-1101] Packing variable 'from_app.V' (/home/user/galapagos/middleware/hls/ne

INFO: [SYSC 207-301] Generating SystemC RTL for raw_bridge.
INFO: [VHDL 208-304] Generating VHDL RTL for raw_bridge.
INFO: [VLOG 209-307] Generating Verilog RTL for raw_bridge.
INFO: [IMPL 213-8] Exporting RTL as a Vivado IP.
INFO: [IMPL 213-200] Port 'from_app_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'from_app_V.dest' is mapped to 'TDEST' by default.
INFO: [IMPL 213-200] Port 'from_app_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'from_app_V.id' is mapped to 'TID' by default.
INFO: [IMPL 213-200] Port 'from_app_V.keep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-200] Port 'from_raw_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'from_raw_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'from_raw_V.tkeep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-200] Port 'to_app_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'to_app_V.dest' is mapped to 'TDEST' by default.
INFO: [IMPL

INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-42] -- Implementing module 'eth_to_app'
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [SCHED 204-11] Starting scheduling ...
INFO: [SCHED 204-61] Pipelining function 'eth_to_app'.
INFO: [SCHED 204-61] Pipelining result : Target II = 1, Final II = 1, Depth = 2.
	fifo read on port 'mac_address_V' (1.75 ns)
	'icmp' operation ('tmp_i', /home/user/galapagos/middleware/hls/network_bridge_ethernet/src/ethernet_bridge.cpp:92) (1.39 ns)
	'select' operation ('storemerge1_cast_i_c', /home/user/galapagos/middleware/hls/network_bridge_ethernet/src/ethernet_bridge.cpp:92) (0.19 ns)
	'store' operation (/home/user/galapagos/middleware/hls/network_bridge_ethernet/src/ethernet_bridge.cpp:93) of variable 'storemerge1_cast_i_c', /home/user/galapagos/middleware/hls/network_bridge_ethernet/src/ethernet_bridge.cpp:92 on static variable 'state_V_1' (1 ns)
INFO: [SCHED 

INFO: [IMPL 213-200] Port 'from_app_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'from_app_V.tdest' is mapped to 'TDEST' by default.
INFO: [IMPL 213-200] Port 'from_app_V.tkeep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-200] Port 'from_eth_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'from_eth_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'from_eth_V.tkeep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-200] Port 'to_app_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'to_app_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'to_app_V.tdest' is mapped to 'TDEST' by default.
INFO: [IMPL 213-200] Port 'to_app_V.tkeep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-200] Port 'to_eth_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'to_eth_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'to_eth_V.tkeep' is mapped to 'TKEEP' by default.

******

The previous section makes the router blocks, and two network bridges. One network bridge translates TCP/IP packets into Galapagos packets, and another network bridge translates L2 ethernet packets into Galapagos packets.
The user can select in their cluster description map file their choice of off-chip network communication.

## Compiling User Kernels (Hardware) 

To create a user kernel it is very similar to that of creating the hlsmiddleware in the previous section.
In the following we have an example.

First we set the board we wish to target:

In [2]:
galapagos-update-board adm-8k5-debug

Updating galapagos initialization...


To compile hardware kernels we need to include the Galapagos hardware library found in `<galapagos_home>/middleware/include`

The following is the tcl_script that compiles the hardware kernel:

```tcl
set galapagos_path $::env(GALAPAGOS_PATH)
set board_name $::env(GALAPAGOS_BOARD_NAME)
set part_name $::env(GALAPAGOS_PART)
set src_path_root [pwd] 

cd $galapagos_path/hlsBuild/${board_name}/ip

open_project hls4ml_hcal
set_top hls4ml_hcal
open_solution "solution1"
set_part ${part_name}
add_files $src_path_root/ereg_v1.cpp
add_files $src_path_root/hls4ml_hcal.cpp -cflags "-I $galapagos_path/middleware/include -I $src_path_root -I $src_path_root/weights -I $src_path_root/nnet_utils"
create_clock -period 250MHz -name default
config_interface -expose_global
csynth_design
export_design -format ip_catalog
close_project

quit

```

The key things to note is that the board name, parth and galapagos_path are all taken from environment variables that was set up earlier. Second thing to note is the inclusion of our middleware library.


This kernel is made with our Makefile (that calls the included tcl script) in the following code block:

In [3]:
cd /home/user/HCAL_HLS4ML
make hls

vivado_hls generate_hls.tcl

****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2018.1 (64-bit)
  **** SW Build 2188600 on Wed Apr  4 18:39:19 MDT 2018
  **** IP Build 2185939 on Wed Apr  4 20:55:05 MDT 2018
    ** Copyright 1986-2018 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2018.1/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/opt/Xilinx/Vivado/2018.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'user' on host 'localhost' (Linux_x86_64 version 4.4.0-159-generic) on Fri May 29 19:18:51 +0000 2020
INFO: [HLS 200-10] In directory '/home/user/HCAL_HLS4ML'
INFO: [HLS 200-10] Creating and opening project '/home/user/galapagos/hlsBuild/adm-8k5-debug/ip/hls4ml_hcal'.
INFO: [HLS 200-10] Creating and opening solution '/home/user/galapagos/hlsBuild/adm-8k5-debug/ip/hls4ml_hcal/solution1'.
INFO: [HLS 200-10] Setting target device to 'xcku115-flva1517-2-e'
INFO: [HLS 200-10] Adding design file '/home/user/HCAL_HLS4ML/ereg_v1

INFO: [XFORM 203-501] Unrolling loop 'Result' (/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:119) in function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely.
INFO: [XFORM 203-501] Unrolling loop 'Loop-1' (/home/user/HCAL_HLS4ML/nnet_utils/nnet_activation.h:57) in function 'nnet::linear<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, linear_config1>' completely.
INFO: [XFORM 203-501] Unrolling loop 'Product1' (/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:86) in function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config1>' completely.
INFO: [XFORM 203-501] Unrolling loop 'Product2' (/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:91) in function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config1>

INFO: [XFORM 203-622] Instantiating function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config3>.0'(/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:62:1) to 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config3>.0.0' at call site (/home/user/HCAL_HLS4ML/ereg_v1.cpp:76) by setting 'biases.V.read' to '-58281'.
INFO: [XFORM 203-622] Instantiating function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>'(/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:62:13) to 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>.0' at call site (/home/user/HCAL_HLS4ML/ereg_v1.cpp:71) by setting 'weights.V' to 'w2'.
INFO: [XFORM 203-622] Instantiating function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5

INFO: [XFORM 203-622] Instantiating function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config1>.0.0.0.0.0.0.0.0.0.0.0'(/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:62:28) to 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config1>.0.0.0.0.0.0.0.0.0.0.0.0' at call site (/home/user/HCAL_HLS4ML/ereg_v1.cpp:64) by setting 'biases[10].V' to 'b1.10'.
INFO: [XFORM 203-622] Instantiating function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config1>.0.0.0.0.0.0.0.0.0.0.0.0'(/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:62:28) to 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config1>.0.0.0.0.0.0.0.0.0.0.0.0.0' at call site (/home/user/HCAL_HLS4ML/ereg_v1.cpp:64) by setting 'biases[11].V' to 'b1.11'.
IN

INFO: [HLS 200-111]  Elapsed time: 0.03 seconds; current allocated memory: 403.601 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-42] -- Implementing module 'compute_layer_0_0_0_s'
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [SCHED 204-11] Starting scheduling ...
INFO: [SCHED 204-61] Pipelining function 'compute_layer.0.0.0.'.
INFO: [SCHED 204-61] Pipelining result : Target II = 1, Final II = 1, Depth = 5.
INFO: [SCHED 204-11] Finished scheduling.
INFO: [HLS 200-111]  Elapsed time: 1.69 seconds; current allocated memory: 406.015 MB.
INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIND 205-101] Performing variable lifetime analysis.
INFO: [BIND 205-101] Exploring resource sharing.
INFO: [BIND 205-101] Binding ...
INFO: [BIND 205-100] Starting global binding ...
INFO: [BIND 205-100] Finished micro-architecture generation.
INFO: [HLS 200-111]  Elapsed time: 0.64 sec

INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_mul_19s_32s_50_3_1' to 'hls4ml_hcal_mul_1vdy' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_mul_5ns_32s_37_3_1' to 'hls4ml_hcal_mul_5wdI' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_mul_16s_32s_47_3_1' to 'hls4ml_hcal_mul_1xdS' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_mul_17s_32s_48_3_1' to 'hls4ml_hcal_mul_1yd2' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_mul_23ns_32s_50_3_1' to 'hls4ml_hcal_mul_2zec' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_mul_18s_32s_49_3_1' to 'hls4ml_hcal_mul_1Aem' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_mul_19ns_32s_50_3_1' to 'hls4ml_hcal_mul_1Bew' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_mul_24ns_32s_50_3_1' to 'hls4ml_hcal_mul_2CeG' due to the lengt

INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3Thq': 3 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3UhA': 1 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3VhK': 1 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3WhU': 5 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3Xh4': 3 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3Yie': 1 instance(s).
INFO: [RTGEN 206-100] Finished creating RTL model for 'compute_layer_0_0_0_s'.
INFO: [HLS 200-111]  Elapsed time: 0.82 seconds; current allocated memory: 450.914 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-10] -- Generating RTL for module 'linear_1'
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [RTGEN 206-100] Finished creating RTL model for 'linear_1'.
INFO: [HLS 200-111]  Elapsed time: 2.

INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_out_buf_4_V' to 'hls4ml_hcal_out_bbyn' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_out_buf_5_V' to 'hls4ml_hcal_out_bbzo' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_out_buf_6_V' to 'hls4ml_hcal_out_bbAo' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_out_buf_7_V' to 'hls4ml_hcal_out_bbBo' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_out_buf_8_V' to 'hls4ml_hcal_out_bbCo' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_out_buf_9_V' to 'hls4ml_hcal_out_bbDo' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_out_buf_10_V' to 'hls4ml_hcal_out_bbEo' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_out_buf_11_V' to 'hls4ml_hcal_out_bbFp' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4m

INFO: [IP_Flow 19-1704] No user IP repositories specified
INFO: [IP_Flow 19-2313] Loaded Vivado IP repository '/opt/Xilinx/Vivado/2018.1/data/ip'.
INFO: [Common 17-206] Exiting Vivado at Fri May 29 19:21:29 2020...
INFO: [HLS 200-112] Total elapsed time: 158.22 seconds; peak allocated memory: 467.537 MB.
INFO: [Common 17-206] Exiting vivado_hls at Fri May 29 19:21:29 2020...


This will place the generated IP cores in `<galapagos_home>/hlsBuild/<board_name>/ip`. Here we can see all the IP cores that we have created for this board. This includes the middleware IP cores and the User IP cores

In [4]:
cd $GALAPAGOS_PATH/hlsBuild/$GALAPAGOS_BOARD_NAME/ip
ls

ethernet_bridge   network_bridge_ethernet  raw_bridge
galapagos_bridge  network_bridge_raw       width32
hls4ml_hcal       network_bridge_tcp       width48


Now that we have created our IP cores, we can describe them in a cluster and stitch them together. <a href='http://127.0.0.1:9000/notebooks/02_middleware.ipynb'>Using the Middleware'</a>

## Compiling User Kernels (Software)

Hardware and software kernels have the same code structure (HLS code). However the compilation process is slightly different. To compile software we have to include the Galapagos library found in: `<galapagos_home>/middleware/CPP_lib/Galapagos_lib`

This is compiled using our Makefile in the following code block:

In [6]:
cd /home/user/HCAL_HLS4ML
make cpu_node.exe

g++ -DCPU -O2 -std=c++17 -pthread -isystem /opt/Xilinx/Vivado/2018.1/include -I/home/user/galapagos/middleware/include -I/home/user/galapagos/middleware/libGalapagos -I/home/user/galapagos/util/spdlog/include -I../nnet_utils -c ereg_v1.cpp -o ereg_v1.o -lboost_thread -lboost_system -lpthread 
g++ -DCPU -O2 -std=c++17 -pthread -isystem /opt/Xilinx/Vivado/2018.1/include -I/home/user/galapagos/middleware/include -I/home/user/galapagos/middleware/libGalapagos -I/home/user/galapagos/util/spdlog/include -I../nnet_utils -c hls4ml_hcal.cpp -o hls4ml_hcal.o -lboost_thread -lboost_system -lpthread 
g++ -DCPU -O2 -std=c++17 -pthread -isystem /opt/Xilinx/Vivado/2018.1/include -I/home/user/galapagos/middleware/include -I/home/user/galapagos/middleware/libGalapagos -I/home/user/galapagos/util/spdlog/include -I../nnet_utils -I/home/user/galapagos/middleware/libGalapagos -I/home/user/galapagos/util/argparse -DCPU -DLOG_LEVEL=0 ereg_v1.o hls4ml_hcal.o cpu_node.cpp -o cpu_node.exe -lboost_thread -lboost

## Example Galapagos Kernel

Since both hardware and software kernels look the same we can investigate both kernels with one example. The following is a code example of a sample kernel:

```cpp
void kern_send(short id, galapagos_stream * in, galapagos_stream  * out)
{
#pragma HLS INTERFACE ap_ctrl_none port=return
#pragma HLS INTERFACE axis register both port=out
#pragma HLS INTERFACE axis register both port=in

    int num_flits = STREAMSIZE*N_INPUTS;

    galapagos_stream_packet gp;
    gp.id = id;

    #include "inputs_demo.h"


    for(int j=0; j<num_flits; j++){
        gp.dest = 1; // FIRST
        gp.data = 0;
        (gp.data)(31,0) = input_vals[j];
        if(j==(num_flits -1))
            gp.last = 1;
        else
            gp.last = 0;
        out->write(gp);
    }   


    num_flits = STREAMSIZE*N_OUTPUTS;


    for(int j=0; j<num_flits; j++){
        gp = in->read();
    }
}

```

### Interfaces

All kernels have 3 interfaces <br/>
1. ID: This is the id of the kernel. If another kernel specifies the dest field of an outgoing packet with this ID then this kernel will receive it in the in stream.
2. In: This is the input galapagos_stream. 
3. Out: This is the output galapagos_stream

At the very beginning there are three interface pragma. This is ignored by gcc if compiling for software.


### Program Behaviour

This program first writes in a loop of flits to dest of 1. It produces one packet with `<num_flits>` flits. The end of the packet is signified by the raising of the last signal.

Then the program waits to receive one multi-flit packet at the end of the program. Both reading and writing of these streams are blocking calls.